In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta, datetime
from time import sleep

import pandas as pd
import csv
from tqdm import tqdm

In [ ]:
start_date = datetime(2017, 6, 1)
end_date = datetime(2018, 8, 12)
days_between = (end_date - start_date).days
dates = [start_date + timedelta(days=i) for i in range(days_between + 1)]
dates

In [6]:
csv_file = "news_data.csv"

with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "title", "description"])
    writer.writeheader()

    for dt in tqdm(dates):
        data = []
        response = requests.get('https://cafef.vn/thi-truong-chung-khoan/' + dt.strftime('%d/%m/%Y') + '.chn')
        soup = BeautifulSoup(response.content, 'html.parser')
        
        titles = soup.find_all('h3')
        des = soup.find_all('p', {'class': 'sapo'})
        
        for t, d in zip(titles, des):
            row = {}
            row['title'] = t.text.strip()
            row['description'] = d.text.strip()
            new_url = t.a.get('href')  # get attribute 'href' from <a> element
            if not new_url.startswith('http'):
                new_url = 'https://cafef.vn' + new_url
        
            response = requests.get(new_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            time = soup.find('span', {'class': 'pdate'})
            try:
                row['date'] = time.text.strip()
            except:
                row['date'] = dt.strftime('%d/%m/%Y')
            data.append(row)
        
        # Write data for each day to the CSV file
        for row in data:
            writer.writerow(row)

  0%|          | 0/438 [00:00<?, ?it/s]

100%|██████████| 438/438 [32:38<00:00,  4.47s/it]
